In [1]:
pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=cbc818c25b600c7450a83f26dabc1d5e1c9f41febfc582a4145fe7a1a37b7351
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry


In [4]:
import pandas as pd
import pycountry

In [5]:
#leemos la base de datos en este caso es WDI
data = pd.read_csv('/content/World Development Indicators.csv')

In [6]:
#creamos un codigo por pais
y = 0
data['cod'] = 0
l = []
for i in range(len(data)):
  if data['Country Code'].iloc[i] not in l:
    y += 1
    l.append(data['Country Code'].iloc[i])
    data['cod'].iloc[i] = y
  else:
    data['cod'].iloc[i] = y

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [144]:
# importamos los datos del top 16
df_1990_or = pd.read_csv('/content/top_16_origen_2000.csv')
df_1990_de = pd.read_csv('/content/top_16_destino_2000.csv')
# df_1995_de = pd.read_csv('/content/top_16_destino_1995.csv')
# df_1995_or = pd.read_csv('/content/top_16_origen_1995.csv')
# df_2000_de = pd.read_csv('/content/top_16_destino_2000.csv')
# df_2000_or = pd.read_csv('/content/top_16_origen_2000.csv')
# df_2005_de = pd.read_csv('/content/top_16_destino_2005.csv')
# df_2005_or = pd.read_csv('/content/top_16_origen_2005.csv')
# df_2010_de = pd.read_csv('/content/top_16_destino_2010.csv')
# df_2010_or = pd.read_csv('/content/top_16_origen_2010.csv')
# df_2015_de = pd.read_csv('/content/top_16_destino_2015.csv')
# df_2015_or = pd.read_csv('/content/top_16_origen_2015.csv')
# df_2020_de = pd.read_csv('/content/top_16_destino_2020.csv')
# df_2020_or = pd.read_csv('/content/top_16_origen_2020.csv')
# df_promedio = pd.read_csv('/content/top_16_origen_promedio.csv')

In [155]:
# aplicamos una serie de normalizaciones para una posterios homologacion
df_1990_de['Region, development group, country or area'] = df_1990_de['Region, development group, country or area'].str.strip()
df_1990_or['Region, development group, country or area'] = df_1990_or['Region, development group, country or area'].str.strip()
df_1990_de['Region, development group, country or area'] = df_1990_de['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})
df_1990_or['Region, development group, country or area'] = df_1990_or['Region, development group, country or area'].replace({"Côte d'Ivoire" : "cote d'ivoire"})

In [156]:
#vemos los datos que manejamos
df_1990_or['Region, development group, country or area'].unique()

array(['united states', 'russian federation', 'germany', 'india',
       'france', 'ukraine', 'canada', 'saudi arabia', 'united kingdom',
       'australia', 'pakistan', 'kazakhstan', 'hong kong', 'iran',
       'united arab emirates', "côte d'ivoire"], dtype=object)

In [157]:
# aplicamos la normalizacion, de existir casos particulares pueden ser agregados al diccionario custom_mapping o a except LookupError
def normalize_country(name):

    custom_mapping = {
        "Iran (Islamic Republic of)": "iran",
        "côte d'ivoire": "cote d'ivoire",
        "China, Hong Kong SAR*": "hong kong",
        "hong kong sar, china": "hong kong",
        "iran, islamic rep.": "iran"
    }
    
    if name in custom_mapping:
        return custom_mapping[name]
    try:
        country = pycountry.countries.lookup(name)
        return country.name
    except LookupError:
        if name == "United States of America*":
            return "United States"
        elif name == "Iran (Islamic Republic of)":
            return "iran"
        elif name == "Russian Federation":
            return "Russia"
        elif name == "France*":
            return "France"
        elif name == "Australia*":
            return "Australia"
        elif name == "United Kingdom*":
            return "United Kingdom"
        elif name == "China, Hong Kong SAR*":
            return "hong kong"                 
        elif name == "Ukraine*":
            return "Ukraine"
        # agrega más verificaciones aquí para otros casos especiales
        else:
          return name

df_1990_de['Region, development group, country or area'] = df_1990_de['Region, development group, country or area'].apply(normalize_country).str.lower()
df_1990_or['Region, development group, country or area'] = df_1990_or['Region, development group, country or area'].apply(normalize_country).str.lower()
data['Country Name'] = data['Country Name'].apply(normalize_country).str.lower()

In [158]:
# validacion de datos
name_de_1990 = list(df_1990_de['Region, development group, country or area'].unique())
name_or_1990 = list(df_1990_or['Region, development group, country or area'].unique())
# name_de_1990 = list(map(lambda x: x.strip().lower(), name_de_1990))
# name_or_1990 = list(map(lambda x: x.strip().lower(), name_or_1990))

In [159]:
#forma de saber si se ingestaron todos los datos o si hay alguna incopatibilida
not_in_data = []
for name in name_de_1990:
  if name not in data['Country Name'].values:
    not_in_data.append(name)

print(not_in_data)


[]


In [ ]:
# data['Country Name'].str.contains('iran').sum()
# data['Country Name'].str.contains("cote d'ivoire").sum
# data['Country Name'].unique()
# data['Country Name'].unique()

In [143]:
#Creamos la mascara y a su vez el archivo csv
# name_de_1990 = list(map(lambda x: x.strip().lower(), name_de_1990))
# name_or_1990 = list(map(lambda x: x.strip().lower(), name_or_1990))

# filtro_or = data[data['Country Name'].isin(df_1990_or['Region, development group, country or area'])]
filtro_or = data[(data['Country Name'].isin(df_1990_or['Region, development group, country or area']))]
filtro_or.to_csv('WDI-top-1995-or.csv', index = False)

filtro_de = data[(data['Country Name'].isin(df_1990_de['Region, development group, country or area']))]
filtro_de.to_csv('WDI-top-1995-de.csv', index = False)

In [113]:
# dic_de = {key: value for key, value in zip(filtro_de['Country Name'], filtro_de['cod'])}
# dic_or = {key: value for key, value in zip(filtro_or['Country Name'], filtro_or['cod'])}
# data_dic =  {key: value for key, value in zip(data['Country Name'], data['cod'])}
dic = data[['Country Name', 'cod']]
dic.drop_duplicates(inplace=True)
dic.to_csv('codigo pais.csv', index = False)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [115]:
filtro_or

,Unnamed: 0,Country Name,Country Code,Series Name,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],cod
1030,1030,australia,aus,access to electricity (% of population),1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,...,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,11
1031,1031,australia,aus,adjusted net national income (current us$),2.414435e+11,2.533194e+11,2.534232e+11,2.468978e+11,2.579116e+11,2.891572e+11,...,1.213570e+12,1.245098e+12,1.142430e+12,1.054276e+12,9.447817e+11,1.032978e+12,1.098286e+12,1.073529e+12,1.031357e+12,11
1032,1032,australia,aus,adjusted net national income per capita (annua...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,11
1033,1033,australia,aus,adjusted savings: gross savings (% of gni),2.519037e+01,2.155649e+01,1.891799e+01,2.073196e+01,2.222046e+01,2.138795e+01,...,2.514171e+01,2.458846e+01,2.430342e+01,2.317352e+01,2.121822e+01,2.241241e+01,2.244137e+01,2.327337e+01,2.457499e+01,11
1034,1034,australia,aus,"adolescent fertility rate (births per 1,000 wo...",2.096500e+01,2.092000e+01,2.087500e+01,2.059060e+01,2.030620e+01,2.002180e+01,...,1.230200e+01,1.218460e+01,1.206720e+01,1.194980e+01,1.183240e+01,1.171500e+01,1.132580e+01,1.093660e+01,1.054740e+01,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21522,21522,uzbekistan,uzb,"unemployment, total (% of total labor force) (...",0.000000e+00,0.000000e+00,2.000000e-01,4.000000e-01,4.000000e-01,4.000000e-01,...,4.870000e+00,4.860000e+00,5.090000e+00,5.150000e+00,5.160000e+00,5.830000e+00,9.350000e+00,8.980000e+00,5.290000e+00,209
21523,21523,uzbekistan,uzb,"unemployment, total (% of total labor force) (...",0.000000e+00,1.900000e+00,2.900000e+00,4.900000e+00,7.300000e+00,7.800000e+00,...,4.900000e+00,4.900000e+00,5.100000e+00,5.200000e+00,5.200000e+00,5.800000e+00,5.835000e+00,5.850000e+00,7.041000e+00,209
21524,21524,uzbekistan,uzb,urban population (% of total population),4.136500e+01,4.183600e+01,4.230800e+01,4.278000e+01,4.325500e+01,4.373100e+01,...,5.105000e+01,5.095000e+01,5.085000e+01,5.075000e+01,5.065000e+01,5.055000e+01,5.047800e+01,5.043300e+01,5.041600e+01,209
21525,21525,uzbekistan,uzb,urban population,8.483962e+06,8.765479e+06,9.074643e+06,9.386788e+06,9.679171e+06,9.964108e+06,...,1.519988e+07,1.540891e+07,1.564029e+07,1.588419e+07,1.613096e+07,1.637244e+07,1.663558e+07,1.693558e+07,1.725843e+07,209


In [92]:
filtro_de

,Unnamed: 0,Country Name,Country Code,Series Name,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],cod
1030,1030,australia,aus,access to electricity (% of population),1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,...,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,11
1031,1031,australia,aus,adjusted net national income (current us$),2.414435e+11,2.533194e+11,2.534232e+11,2.468978e+11,2.579116e+11,2.891572e+11,...,1.213570e+12,1.245098e+12,1.142430e+12,1.054276e+12,9.447817e+11,1.032978e+12,1.098286e+12,1.073529e+12,1.031357e+12,11
1032,1032,australia,aus,adjusted net national income per capita (annua...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,11
1033,1033,australia,aus,adjusted savings: gross savings (% of gni),2.519037e+01,2.155649e+01,1.891799e+01,2.073196e+01,2.222046e+01,2.138795e+01,...,2.514171e+01,2.458846e+01,2.430342e+01,2.317352e+01,2.121822e+01,2.241241e+01,2.244137e+01,2.327337e+01,2.457499e+01,11
1034,1034,australia,aus,"adolescent fertility rate (births per 1,000 wo...",2.096500e+01,2.092000e+01,2.087500e+01,2.059060e+01,2.030620e+01,2.002180e+01,...,1.230200e+01,1.218460e+01,1.206720e+01,1.194980e+01,1.183240e+01,1.171500e+01,1.132580e+01,1.093660e+01,1.054740e+01,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21522,21522,uzbekistan,uzb,"unemployment, total (% of total labor force) (...",0.000000e+00,0.000000e+00,2.000000e-01,4.000000e-01,4.000000e-01,4.000000e-01,...,4.870000e+00,4.860000e+00,5.090000e+00,5.150000e+00,5.160000e+00,5.830000e+00,9.350000e+00,8.980000e+00,5.290000e+00,209
21523,21523,uzbekistan,uzb,"unemployment, total (% of total labor force) (...",0.000000e+00,1.900000e+00,2.900000e+00,4.900000e+00,7.300000e+00,7.800000e+00,...,4.900000e+00,4.900000e+00,5.100000e+00,5.200000e+00,5.200000e+00,5.800000e+00,5.835000e+00,5.850000e+00,7.041000e+00,209
21524,21524,uzbekistan,uzb,urban population (% of total population),4.136500e+01,4.183600e+01,4.230800e+01,4.278000e+01,4.325500e+01,4.373100e+01,...,5.105000e+01,5.095000e+01,5.085000e+01,5.075000e+01,5.065000e+01,5.055000e+01,5.047800e+01,5.043300e+01,5.041600e+01,209
21525,21525,uzbekistan,uzb,urban population,8.483962e+06,8.765479e+06,9.074643e+06,9.386788e+06,9.679171e+06,9.964108e+06,...,1.519988e+07,1.540891e+07,1.564029e+07,1.588419e+07,1.613096e+07,1.637244e+07,1.663558e+07,1.693558e+07,1.725843e+07,209
